In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import statsmodels.api as sm

In [2]:
cars = pd.read_csv("https://stepik.org/media/attachments/lesson/387691/cars.csv")

In [3]:
cars.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [4]:
cars.isnull().sum()

car_ID              0
symboling           0
CarName             0
fueltype            0
aspiration          0
doornumber          0
carbody             0
drivewheel          0
enginelocation      0
wheelbase           0
carlength           0
carwidth            0
carheight           0
curbweight          0
enginetype          0
cylindernumber      0
enginesize          0
fuelsystem          0
boreratio           0
stroke              0
compressionratio    0
horsepower          0
peakrpm             0
citympg             0
highwaympg          0
price               0
dtype: int64

**Using the full name of the car is not the best one, so let's create a new variant - the brand of the car (company). Break the cell values of CarName column by space and write the first element in the column. For example: 'audi 100 ls' → 'audi'**    

**How many unique car brands are there in the dataset?**

In [5]:
cars["company"] = cars.CarName.apply(lambda x: x.split()[0])

In [6]:
cars.company.nunique()

28

In [7]:
cars.drop(['CarName', 'car_ID'], axis=1, inplace=True)

In [8]:
cars.head()

,symboling,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,...,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price,company
0,3,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,...,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0,alfa-romero
1,3,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,...,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0,alfa-romero
2,1,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,...,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0,alfa-romero
3,2,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,...,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0,audi
4,2,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,...,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0,audi


**Now take a closer look at the unique values of `company`. Some of them turned out to be with errors!**   

* 'maxda' → 'mazda'
* 'Nissan' → 'nissan'
* 'porcshce' → 'porsche'
* 'toyouta' → 'toyota'
* 'vokswagen' & 'vw' → 'volkswagen'

**How many unique companies are left as a result?**

In [9]:
cars["company"] = cars.company.replace(to_replace='maxda', value='mazda').replace('porcshce', "porsche").replace('toyouta', 'toyota') \
    .replace(['vokswagen', 'vw'], 'volkswagen').str.lower()

In [10]:
cars.company.nunique()

22

**In order not to overload the model with a large number of predictors, we will leave only some of them:**    

**'company', 'fueltype', 'aspiration', 'carbody', 'drivewheel', 'wheelbase', 'carlength', 'carwidth', 'curbweight', 'enginetype', 'cylindernumber', 'enginesize', 'boreratio ',' horsepower '    
also not forgetting what we predict - 'price'.**    

**Then calculate the correlation between price and other variables.**

In [11]:
cars_new = cars[cars.columns[cars.columns.isin(['company', 'fueltype', 'aspiration','carbody', 'drivewheel', 'wheelbase', 'carlength','carwidth', 'curbweight', 'enginetype', 'cylindernumber', 'enginesize', 'boreratio','horsepower', 'price'])]]

In [12]:
cars_new.corr().round(2)

,wheelbase,carlength,carwidth,curbweight,enginesize,boreratio,horsepower,price
wheelbase,1.00,0.87,0.80,0.78,0.57,0.49,0.35,0.58
carlength,0.87,1.00,0.84,0.88,0.68,0.61,0.55,0.68
carwidth,0.80,0.84,1.00,0.87,0.74,0.56,0.64,0.76
curbweight,0.78,0.88,0.87,1.00,0.85,0.65,0.75,0.84
enginesize,0.57,0.68,0.74,0.85,1.00,0.58,0.81,0.87
boreratio,0.49,0.61,0.56,0.65,0.58,1.00,0.57,0.55
horsepower,0.35,0.55,0.64,0.75,0.81,0.57,1.00,0.81
price,0.58,0.68,0.76,0.84,0.87,0.55,0.81,1.00


**The last step in preparing the data: linear regression in python doesn't handle categorical variables (the object type in pandas), so let's use pd.get_dummies()**    
**How many columns are there in the dataset now?**

In [13]:
cars_new.select_dtypes(include=object).columns

Index(['fueltype', 'aspiration', 'carbody', 'drivewheel', 'enginetype',
       'cylindernumber', 'company'],
      dtype='object')

In [14]:
cars_dummy = pd.get_dummies(data=cars_new[['fueltype', 'aspiration', 'carbody', 'drivewheel', 'enginetype',
       'cylindernumber', 'company']], drop_first=True)

In [15]:
cars_dummy.head()

,fueltype_gas,aspiration_turbo,carbody_hardtop,carbody_hatchback,carbody_sedan,carbody_wagon,drivewheel_fwd,drivewheel_rwd,enginetype_dohcv,enginetype_l,...,company_nissan,company_peugeot,company_plymouth,company_porsche,company_renault,company_saab,company_subaru,company_toyota,company_volkswagen,company_volvo
0,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
cars_new.select_dtypes(include=[float, int]).columns

Index(['wheelbase', 'carlength', 'carwidth', 'curbweight', 'enginesize',
       'boreratio', 'horsepower', 'price'],
      dtype='object')

In [17]:
cars_for_model = pd.concat([cars_dummy, cars_new[['wheelbase', 'carlength', 'carwidth', 'curbweight', 'enginesize',
       'boreratio', 'horsepower', 'price']]], axis=1)

In [18]:
cars_for_model.shape

(205, 49)

**First, let's build a simple model with just one price predictor - horsepower.**

**What percentage of variability does the resulting model explain?**

In [19]:
results = ols('price ~ horsepower', cars_for_model).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.653
Model:                            OLS   Adj. R-squared:                  0.651
Method:                 Least Squares   F-statistic:                     382.2
Date:                Mon, 25 Oct 2021   Prob (F-statistic):           1.48e-48
Time:                        14:46:38   Log-Likelihood:                -2024.0
No. Observations:                 205   AIC:                             4052.
Df Residuals:                     203   BIC:                             4059.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -3721.7615    929.849     -4.003      0.0

In [20]:
# 65%

**Now two models:**

* model with all predictors
* model with all predictors except car company    

**Note the changes in $R^2$, coefficients and their significance. Which model is better to keep?**

In [21]:
cars_for_model.columns

Index(['fueltype_gas', 'aspiration_turbo', 'carbody_hardtop',
       'carbody_hatchback', 'carbody_sedan', 'carbody_wagon', 'drivewheel_fwd',
       'drivewheel_rwd', 'enginetype_dohcv', 'enginetype_l', 'enginetype_ohc',
       'enginetype_ohcf', 'enginetype_ohcv', 'enginetype_rotor',
       'cylindernumber_five', 'cylindernumber_four', 'cylindernumber_six',
       'cylindernumber_three', 'cylindernumber_twelve', 'cylindernumber_two',
       'company_audi', 'company_bmw', 'company_buick', 'company_chevrolet',
       'company_dodge', 'company_honda', 'company_isuzu', 'company_jaguar',
       'company_mazda', 'company_mercury', 'company_mitsubishi',
       'company_nissan', 'company_peugeot', 'company_plymouth',
       'company_porsche', 'company_renault', 'company_saab', 'company_subaru',
       'company_toyota', 'company_volkswagen', 'company_volvo', 'wheelbase',
       'carlength', 'carwidth', 'curbweight', 'enginesize', 'boreratio',
       'horsepower', 'price'],
      dtype='object'

In [22]:
predictors_full = cars_for_model.drop(["price"], axis=1)

In [23]:
X = sm.add_constant(predictors_full)

model = sm.OLS(cars_for_model.price, X) 
results = model.fit()  
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.959
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     81.09
Date:                Mon, 25 Oct 2021   Prob (F-statistic):           4.86e-89
Time:                        14:46:38   Log-Likelihood:                -1804.2
No. Observations:                 205   AIC:                             3702.
Df Residuals:                     158   BIC:                             3858.
Df Model:                          46                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                 -3.472e+

In [24]:
predictors_wo_company = cars_for_model[cars_for_model.columns.drop(list(cars_for_model.filter(regex='company')))].drop(['price'], axis=1)

In [25]:
X = sm.add_constant(predictors_wo_company) 

model = sm.OLS(cars_for_model.price, X) 
results = model.fit() 
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.914
Model:                            OLS   Adj. R-squared:                  0.901
Method:                 Least Squares   F-statistic:                     72.32
Date:                Mon, 25 Oct 2021   Prob (F-statistic):           9.86e-81
Time:                        14:46:38   Log-Likelihood:                -1881.6
No. Observations:                 205   AIC:                             3817.
Df Residuals:                     178   BIC:                             3907.
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   -1.7e+

We leave the model with fewer predictors, because R2 has not changed very much, and some companies are not significant at all.    
The chosen model explains approximately 90 % variance. Among predictors 10 of 27 were not significant (p > 0.05).     
Interpretation example: for a single change in horsepower, the price increases on 86.8164.